In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here.
bucket = sess.default_bucket()

# Location to put your custom code.
custom_code_upload_location = "customcode"

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# You can use the Amazon SageMaker Python SDK to get the role from the notebook environment.
role = get_execution_role()

In [5]:
from sagemaker.pytorch import PyTorch

CODE_PATH = "gcn.py"
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

params = {'epochs' : 100, 'runs' : 1}

task_tags = [{"Key": "ML Task", "Value": "DGL"}]
estimator = PyTorch(
    entry_point=CODE_PATH,
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",  # 'ml.c4.2xlarge '
    framework_version="1.8.0",
    py_version="py3",
    debugger_hook_config=False,
    tags=task_tags,
    hyperparameters=params,
    sagemaker_session=sess,
)

In [6]:
estimator.fit()

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.g4dn.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

In [ ]:
predictor = pytorch_estimator.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1)

In [ ]:
response = predictor.predict(data)

In [ ]:
# ?
dataset = PygNodePropPredDataset(name='ogbn-arxiv',
                                     transform=T.ToSparseTensor())

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()
data = data.to(device)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)


out = model(data.x, data.adj_t)
y_pred = out.argmax(dim=-1, keepdim=True)

train_acc = evaluator.eval({
    'y_true': data.y[split_idx['train']],
    'y_pred': y_pred[split_idx['train']],
})['acc']
valid_acc = evaluator.eval({
    'y_true': data.y[split_idx['valid']],
    'y_pred': y_pred[split_idx['valid']],
})['acc']
test_acc = evaluator.eval({
    'y_true': data.y[split_idx['test']],
    'y_pred': y_pred[split_idx['test']],
})['acc']

return train_acc, valid_acc, test_acc